# Tutorial for infer_structcol package

This tutorial explains how to use the infer_structcol package to infer the volume fraction of a structurally colored sample using experimental reflectance and transmittance data. 

In [1]:
# standard imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import seaborn as sns

# import package
import infer_structcol as ifs

## Set parameters

In [2]:
nwalkers = 10                                    # number of walkers to step through different parameters
nsteps = 100                                    # number of steps for each walker to take
particle_radius = 119                           # radius of colloidal particles in the colloidal glass (in nm)
thickness = 120                                 # thickness of colloidal glass film (in um)
particle_index = 1.59                           # refractive index of colloidal particles 
matrix_index = 1                                # refractive index of matrix material surrounding particles
wavelengths = [450,500,550,600,650,700,750,800] # wavelengths of interest where spectrum should be calculated (in nm)

# Initial guess for the parameters to be inferred: theta_guess = (volume fraction, loss intercept for reflectance, 
# loss slope for reflectance, loss intercept for transmittance, loss slope for transmittance). If only reflectance or 
# transmittance data are used, then theta_guess = (volume fraction, loss intercept, loss slope). 
theta_guess = (0.59, 0.5, 0, 0.5, 0)
directory = os.path.join('infer_structcol','tests','test_data','experimental_data')
#directory = 'infer_structcol'                   # path where the data is found
refl_reference_file = 'R_ref.txt'                      # name of reference file for reflectance data
refl_dark_file = 'R_dark.txt'                          # name of dark file for reflectance data
trans_reference_file = 'T_ref.txt'                      # name of reference file for transmittance data
trans_dark_file = 'T_dark.txt'                          # name of dark file for transmittance data

## Infer parameters
##### Convert raw data into a normalized spectrum and load it
All data files for transmission (including reference and dark measurements) should be located in a directory called 'transmission', and all data files for reflection should be located in a directory called 'reflection'. They should be .txt files with two $\textit{tab-separated}$ columns, where the first is the wavelength and the second is the reflectance or transmittance fraction. Any header information at the top or bottom of the file will be ignored. The reference and dark spectra should have unique names as specified by the user, and the transmission or reflection spectra can have any unspecified name. The code will assume that any file in the data directory other than the reference or dark files is a transmittance or reflectance spectrum. It is expected that the user will include several reflectance/transmittance measurements of the same sample in order to calculate an uncertainty on the data. 

The data must first be converted into a normalized spectrum with values from 0 to 1, using the reference and dark spectra. This is accomplished using the convert_data() function. The converted data consists of a file with 3 columns: wavelength, normalized intensity, and uncertainty. The uncertainty is the sample standard deviation calculated from all of the reflectance/transmittance data in the directory.

The converted data must then be loaded using the load_spectrum() function. This function selects the first reflectance and transmittance data set and returns a Spectrum class object, which containts the relevant spectrum data as a Pandas dataframe.  

In [3]:
# create a converted data file for transmittance and reflectance spectra in the directory
ifs.convert_data(wavelengths, refl_reference_file, refl_dark_file, os.path.join(directory, 'reflection'))
ifs.convert_data(wavelengths, trans_reference_file, trans_dark_file, os.path.join(directory, 'transmission'))

# load the converted data file as a Spectrum object
spect = ifs.load_spectrum(refl_filepath = os.path.join(directory, 'reflection', 'converted', '0_data_file.txt'), 
                          trans_filepath = os.path.join(directory, 'transmission', 'converted', '0_data_file.txt'))

##### Define sample object
This object contains the parameters relevant to the colloidal glass sample.

In [4]:
samp = ifs.Sample(spect.wavelength, particle_radius, thickness, particle_index, matrix_index)

##### Perform inference calculation

In [5]:
# time the calculation, if desired
t0 = time.time()

# run the markov chain monte carlo calculation to step through different values of 
# parameters and infer the most likely values
walkers = ifs.run_mcmc(spect, samp, nwalkers, nsteps, theta_guess, seed=2)

# print the time the calculation took
print(time.time()-t0)

NaN value of lnprob for parameters: 
[ 0.38426269  0.68648775 -0.19723877  0.68648775 -0.19723877]


ValueError: lnprob returned NaN.

## Save Results
If you want to have these results for later, you can save the markov chains for each parameter as a text file with a desired name.

In [ ]:
np.savetxt('walkers_chain_vf_nwalk' + str(nwalkers) + '_nstep' + str(nsteps) + '.txt', walkers.chain[:,:,0])
np.savetxt('walkers_chain_refl_l0_nwalk' + str(nwalkers) + '_nstep' + str(nsteps) + '.txt', walkers.chain[:,:,1])
np.savetxt('walkers_chain_refl_l1_nwalk' + str(nwalkers) + '_nstep' + str(nsteps) + '.txt', walkers.chain[:,:,2])
np.savetxt('walkers_chain_trans_l0_nwalk' + str(nwalkers) + '_nstep' + str(nsteps) + '.txt', walkers.chain[:,:,3])
np.savetxt('walkers_chain_trans_l1_nwalk' + str(nwalkers) + '_nstep' + str(nsteps) + '.txt', walkers.chain[:,:,4])

## Visualize Results
##### Load the data
Load the data that you saved to a text file previously.

In [ ]:
vf = np.loadtxt('walkers_chain_vf_nwalk' + str(nwalkers) + '_nstep' + str(nsteps) + '.txt')
refl_l0 = np.loadtxt('walkers_chain_refl_l0_nwalk' + str(nwalkers) + '_nstep' + str(nsteps) + '.txt')
refl_l1 = np.loadtxt('walkers_chain_refl_l1_nwalk' + str(nwalkers) + '_nstep' + str(nsteps) + '.txt')
trans_l0 = np.loadtxt('walkers_chain_trans_l0_nwalk' + str(nwalkers) + '_nstep' + str(nsteps) + '.txt')
trans_l1 = np.loadtxt('walkers_chain_trans_l1_nwalk' + str(nwalkers) + '_nstep' + str(nsteps) + '.txt')

##### Plot the traces
Plot the values for the loss parameters and the volume fraction at each step of the Markov chain.

In [ ]:
fig, (ax_vf, ax_refl_l1, ax_refl_l0, ax_trans_l0, ax_trans_l1) = plt.subplots(3)
ax_vf.set(ylabel='volume fraction')
ax_refl_l0.set(ylabel='reflectance l_0')
ax_refl_l1.set(ylabel='reflectance l_1')
ax_trans_l0.set(ylabel='transmittance l_0')
ax_trans_l1.set(ylabel='transmittance l_1')

for i in range(nwalkers):
    sns.tsplot(vf[i,:], ax=ax_vf)
    sns.tsplot(refl_l0[i,:], ax=ax_refl_l0)
    sns.tsplot(refl_l1[i,:], ax=ax_refl_l1)
    sns.tsplot(trans_l0[i,:], ax=ax_trans_l0)
    sns.tsplot(trans_l1[i,:], ax=ax_trans_l1)

##### Estimate the most likely volume fraction

The most likely volume fraction is found by calculating the median value of the volume fraction traces after cutting off the burn-in time. We calculate a 68% confidence interval using the built-in numpy percentile function.

In [ ]:
# trim off the burn in time, which we observe from the plots above
trim_index = 50
vf = np.ndarray.flatten(vf[:,trim_index:])

# find the median of the volume fractions of the walkers
vf_inferred = np.median(vf)
print("The inferred volume fraction is {:.3f} + {:.3f} - {:.3f}".format(vf_inferred,
                                                                       np.percentile(vf,84)-vf_inferred,
                                                                       vf_inferred - np.percentile(vf,16)))